<a href="https://colab.research.google.com/github/rNEGI1729/Loan-Approval-prediction/blob/main/loan_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('loan-train.csv')

In [ ]:
df.fillna(method='ffill', inplace=True)

<ipython-input-192-e9443599d05e>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [ ]:
df = df.dropna(subset=['LoanAmount'])


In [ ]:
df.drop(columns=['Loan_ID'], inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

X1 = df.drop('Loan_Status', axis=1)  # Drop target column from features
y1 = df['Loan_Status']  # Target column
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [ ]:
X_train1.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
370,Female,No,0,Graduate,Yes,15759,0.0,55.0,360.0,1.0,Semiurban
345,Male,Yes,0,Graduate,No,2499,2458.0,160.0,360.0,1.0,Semiurban
119,Female,No,0,Graduate,No,10408,0.0,259.0,360.0,1.0,Urban
80,Female,No,0,Graduate,No,3846,0.0,111.0,360.0,1.0,Semiurban
24,Male,Yes,1,Graduate,No,3717,2925.0,151.0,360.0,0.0,Semiurban


In [ ]:
from sklearn.preprocessing import StandardScaler

X_train = X_train1.copy()
y_train = y_train1.copy()
X_test = X_test1.copy()
y_test = y_test1.copy()

numerical_features = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']
scaler = StandardScaler()
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test[numerical_features] = scaler.fit_transform(X_test[numerical_features])

# One-hot encoding for categorical features(Non ordinal data)
#df = pd.get_dummies(df, columns=['Gender', 'Married', 'Dependents', 'Self_Employed', 'Property_Area'], drop_first=True)

binary_columns = ['Gender', 'Married', 'Dependents', 'Self_Employed', 'Property_Area','Education']
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for i in binary_columns:
    X_train[i] = label_encoder.fit_transform(X_train[i])
    X_test[i] = label_encoder.fit_transform(X_test[i])

y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)

y_train = pd.Series(y_train)


In [ ]:
X_train1.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
370,Female,No,0,Graduate,Yes,15759,0.0,55.0,360.0,1.0,Semiurban
345,Male,Yes,0,Graduate,No,2499,2458.0,160.0,360.0,1.0,Semiurban
119,Female,No,0,Graduate,No,10408,0.0,259.0,360.0,1.0,Urban
80,Female,No,0,Graduate,No,3846,0.0,111.0,360.0,1.0,Semiurban
24,Male,Yes,1,Graduate,No,3717,2925.0,151.0,360.0,0.0,Semiurban


In [ ]:
X_train.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
370,0,0,0,0,1,1.866729,-0.522847,-1.035769,0.273281,0.451587,1
345,1,1,0,0,0,-0.496330,0.260594,0.133417,0.273281,0.451587,1
119,0,0,0,0,0,0.913129,-0.522847,1.235792,0.273281,0.451587,2
80,0,0,0,0,0,-0.256282,-0.522847,-0.412203,0.273281,0.451587,1
24,1,1,1,0,0,-0.279271,0.409441,0.033201,0.273281,-2.214411,1


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define model
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary classification output

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Train model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.3221 - loss: 0.7386 - val_accuracy: 0.3496 - val_loss: 0.7222
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4244 - loss: 0.7089 - val_accuracy: 0.5285 - val_loss: 0.7003
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5462 - loss: 0.6831 - val_accuracy: 0.5935 - val_loss: 0.6867
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6353 - loss: 0.6723 - val_accuracy: 0.6504 - val_loss: 0.6754
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7410 - loss: 0.6574 - val_accuracy: 0.6667 - val_loss: 0.6667
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7839 - loss: 0.6469 - val_accuracy: 0.6829 - val_loss: 0.6591
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8028 - loss: 0.6447 - val_accuracy: 0.7073 - val_loss: 0.6524
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7829 - loss: 0.6378 - val_accuracy: 0.7073 - val_loss

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8238 - loss: 0.5544 
Test Accuracy: 78.86%


In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

# Create your preprocessor pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']),
        ('cat', OneHotEncoder(), ['Gender', 'Married', 'Dependents', 'Self_Employed', 'Property_Area','Education'])
    ])

# Fit the preprocessor to the training data
preprocessor.fit(X_train)  # Replace X_train with your actual training data


ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['ApplicantIncome', 'CoapplicantIncome',
                                  'LoanAmount', 'Loan_Amount_Term',
                                  'Credit_History']),
                                ('cat', OneHotEncoder(),
                                 ['Gender', 'Married', 'Dependents',
                                  'Self_Employed', 'Property_Area',
                                  'Education'])])

In [ ]:
import joblib

# Save the model
joblib.dump(model, 'loan_model.pkl')



# Save the model and the scaler
model.save('loan_model.keras')  # Save Keras model
joblib.dump(scaler, 'scaler.pkl')  # Save the scaler
joblib.dump(label_encoder, 'label_encoder.pkl')  # Save the label encoder
joblib.dump(preprocessor, 'preprocessor.pkl')

['preprocessor.pkl']

In [ ]:
X_train1.head(10)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
370,Female,No,0,Graduate,Yes,15759,0.0,55.0,360.0,1.0,Semiurban
345,Male,Yes,0,Graduate,No,2499,2458.0,160.0,360.0,1.0,Semiurban
119,Female,No,0,Graduate,No,10408,0.0,259.0,360.0,1.0,Urban
80,Female,No,0,Graduate,No,3846,0.0,111.0,360.0,1.0,Semiurban
24,Male,Yes,1,Graduate,No,3717,2925.0,151.0,360.0,0.0,Semiurban
154,Male,No,0,Graduate,No,3254,0.0,50.0,360.0,1.0,Urban
312,Female,No,0,Graduate,No,2507,0.0,56.0,360.0,1.0,Rural
136,Female,Yes,0,Graduate,No,4583,0.0,84.0,360.0,1.0,Rural
317,Male,Yes,0,Graduate,No,2058,2134.0,88.0,360.0,1.0,Urban
413,Male,Yes,0,Not Graduate,No,2253,2033.0,110.0,360.0,1.0,Rural


In [ ]:
y_train1.shape

(490,)

In [ ]:
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
